In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
tf.compat.v1.disable_eager_execution()

In [ ]:
import os

folder_path = r"C:\Users\suraj\data_visualisation\flowers"
if os.path.exists(folder_path):
    flower_types = os.listdir(folder_path)
    print(flower_types)
else:
    print(f"Folder does not exist: {folder_path}")


Folder does not exist: C:\Users\suraj\data_visualisation\flowers


NameError: name 'flowers' is not defined

In [36]:
import zipfile

zip_path = "flowers.zip"  # Replace with your actual uploaded filename

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("flowers")  # Extracts into a folder named "flowers"


In [37]:
import os

# List top-level contents of the extracted folder
folder_path = "flowers"

print("Top-level files and folders inside 'flowers':")
for item in os.listdir(folder_path):
    print("-", item)


Top-level files and folders inside 'flowers':
- flowers


In [44]:
flowers_folder = "flowers"  # set once at the top

# Use it everywhere
os.listdir(flowers_folder)


['flowers']

In [45]:
import os
import shutil

# Path to the outer and inner folders
outer_folder = "flowers"
inner_folder = os.path.join(outer_folder, "flowers")

# Check that both exist
if os.path.exists(inner_folder):
    # Move everything from inner_folder to outer_folder
    for item in os.listdir(inner_folder):
        s = os.path.join(inner_folder, item)
        d = os.path.join(outer_folder, item)
        shutil.move(s, d)

    # Remove the now-empty inner 'flowers' folder
    os.rmdir(inner_folder)
    print(f"✅ Moved contents up and removed: {inner_folder}")
else:
    print("❌ Inner folder does not exist. Already cleaned?")


✅ Moved contents up and removed: flowers/flowers


In [46]:
flower_types = os.listdir("flowers")
print("Categories:", flower_types)


Categories: ['roses', 'tulips']


In [47]:
import cv2
path = 'flowers//'

In [48]:
im_size = 60

In [49]:
images=[] # name of images in that folder
labels =[]  #name of folder
for i in flower_types: #to enter in folders one byone as we have 2 folders
  data_path = path+str(i)
  filenames = [i for i in os.listdir(data_path) if i.endswith('.jpg')]
  for f in filenames:# this for loop is used to reduce the size of the images in the respective folders
    img= cv2.imread(data_path+'/'+f)
    img= cv2.resize(img, (im_size, im_size))
    images.append(img)
    labels.append(i)


In [50]:
images = np.array(images)
images = images.astype('float32')/255
images

array([[[[0.6       , 0.84313726, 0.8862745 ],
         [0.5921569 , 0.8156863 , 0.85882354],
         [0.5921569 , 0.8392157 , 0.8784314 ],
         ...,
         [0.63529414, 0.8509804 , 0.91764706],
         [0.6627451 , 0.87058824, 0.91764706],
         [0.64705884, 0.87058824, 0.92941177]],

        [[0.6627451 , 0.8745098 , 0.9254902 ],
         [0.62352943, 0.84313726, 0.8862745 ],
         [0.627451  , 0.84705883, 0.8901961 ],
         ...,
         [0.5764706 , 0.8039216 , 0.8784314 ],
         [0.6431373 , 0.85882354, 0.9137255 ],
         [0.61960787, 0.84705883, 0.9137255 ]],

        [[0.6392157 , 0.8627451 , 0.92156863],
         [0.60784316, 0.8509804 , 0.8901961 ],
         [0.61960787, 0.85490197, 0.8980392 ],
         ...,
         [0.6745098 , 0.8784314 , 0.9254902 ],
         [0.63529414, 0.8509804 , 0.9098039 ],
         [0.6392157 , 0.85490197, 0.91764706]],

        ...,

        [[0.6313726 , 0.8352941 , 0.8784314 ],
         [0.64705884, 0.88235295, 0.93333334]

In [51]:
images.shape

(1388, 60, 60, 3)

In [52]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

In [53]:
x =images
y= labels
y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print(y)


[0 0 0 ... 1 1 1]


In [72]:
y=y.reshape(-1,1)
encoder = OneHotEncoder(sparse_output=False)
Y =encoder.fit_transform(y)
Y

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [74]:
print("Length of x:", len(x))
print("Length of Y:", len(Y))
# here the length of the x and y are differnt so in order to use the shuffle function length of both should be same hence that it done in the next step

Length of x: 1388
Length of Y: 5552


In [124]:
min_len = min(len(x), len(Y))
x = x[:min_len]
Y = Y[:min_len]
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
x, Y = shuffle(x, Y, random_state=1)


In [125]:
print("Length of x:", len(x))
print("Length of Y:", len(Y))

Length of x: 1388
Length of Y: 1388


In [135]:
train_x, test_x, train_Y, test_Y = train_test_split(x,Y, test_size= 0.20, random_state =42)

In [136]:
print(train_x.shape)
print(test_x.shape)
print(train_Y.shape)
print(test_Y.shape)


(1110, 60, 60, 3)
(278, 60, 60, 3)
(1110, 2)
(278, 2)


In [137]:
#  for image classification using perceptron u need to transfer the 3 dimentional array of image into one direction which is dim1*dim2*dim3
train_x= np.reshape(train_x, (1110,10800))
print(train_x.shape)
test_x= np.reshape(test_x, (278,10800))
test_x.shape

(1110, 10800)


(278, 10800)

In [150]:
learning_rate = 0.01
training_epoch=1000
n_dim=60
n_class=2

X= tf.compat.v1.placeholder(tf.float32,[None, 10800])
W = tf.Variable(tf.zeros([10800, n_class]))
b = tf.Variable(tf.zeros([n_class]))
y =tf.compat.v1.placeholder(tf.float32,[None,n_class]) #actual y

In [151]:
init =tf.compat.v1.global_variables_initializer()

In [152]:
# functionality of perceptron
pred =tf.nn.softmax(tf.matmul(X,W)+b) # softmax activation function : where probability of each class is found and then class with the highest probability is taken as output/deciding
cost_function= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) #calculating the loss
training_step=tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)  #updates the weight and retrain the algo with givven learning rate

In [153]:
sess= tf.compat.v1.Session()
sess.run(init)
cost_history =[]# blank list to store the error

In [158]:
for epoch in range (training_epoch):
  sess.run(training_step, feed_dict={X:train_x,y:train_Y})
  cost= sess.run(cost_function,feed_dict={X:train_x, y:train_Y})
  cost_history=np.append(cost_history,cost)
  print('epoch:',epoch,"-", "cost:", cost)
pred_y = sess.run(pred, feed_dict ={x:test_x})
#here x is a placeholder and y too so x input we are giving through the train_x data similarly to y

epoch: 0 - cost: 0.619163
epoch: 1 - cost: 0.6285801
epoch: 2 - cost: 0.6189243
epoch: 3 - cost: 0.62835103
epoch: 4 - cost: 0.61868626
epoch: 5 - cost: 0.6281224
epoch: 6 - cost: 0.61844873
epoch: 7 - cost: 0.6278944
epoch: 8 - cost: 0.61821187
epoch: 9 - cost: 0.62766665
epoch: 10 - cost: 0.6179756
epoch: 11 - cost: 0.6274395
epoch: 12 - cost: 0.6177399
epoch: 13 - cost: 0.62721264
epoch: 14 - cost: 0.61750495
epoch: 15 - cost: 0.6269863
epoch: 16 - cost: 0.6172705
epoch: 17 - cost: 0.6267604
epoch: 18 - cost: 0.6170367
epoch: 19 - cost: 0.6265349
epoch: 20 - cost: 0.6168036
epoch: 21 - cost: 0.6263099
epoch: 22 - cost: 0.6165709
epoch: 23 - cost: 0.6260853
epoch: 24 - cost: 0.6163389
epoch: 25 - cost: 0.6258611
epoch: 26 - cost: 0.6161074
epoch: 27 - cost: 0.62563735
epoch: 28 - cost: 0.61587656
epoch: 29 - cost: 0.625414
epoch: 30 - cost: 0.6156463
epoch: 31 - cost: 0.6251911
epoch: 32 - cost: 0.6154166
epoch: 33 - cost: 0.62496847
epoch: 34 - cost: 0.6151874
epoch: 35 - cost: 0.62

TypeError: unhashable type: 'numpy.ndarray'